<a href="https://colab.research.google.com/github/lazybuttrying/TensorFlow-Certificate/blob/main/C1/W4_ImageDataGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

**Things to Note:**
1. When coding the `class myCallback`, Python 3 will run into an error
```python
TypeError: '>' not supported between instances of 'NoneType' and 'float'
```
when using the code
```python
if(logs.get('accuracy')>0.99):
```

For Python 3, use the following equivalent code line

```python
if logs.get('accuracy') is not None and logs.get('accuracy') > 0.99:
```

2. You can run the notebook using TensorFlow 2.5.0

In [ ]:
#!pip install tensorflow==2.5.0

In [ ]:
# this is used in downloading data from the google drive
#!pip install gdown

Below is code with a link to a happy or sad dataset which contains 80 images, 40 happy and 40 sad. 
Create a convolutional neural network that trains to 100% accuracy on these images,  which cancels training upon hitting training accuracy of >.999

Hint -- it will work best with 3 convolutional layers.

In [17]:
import tensorflow as tf
import os
import zipfile

!gdown --id 1NvV6VhmrfU8JDZNoEbwJxwx_6dhyN5bf 

zip_ref = zipfile.ZipFile("./happy-or-sad.zip", 'r')
zip_ref.extractall("./h-or-s")
zip_ref.close()

Downloading...
From: https://drive.google.com/uc?id=1NvV6VhmrfU8JDZNoEbwJxwx_6dhyN5bf
To: /content/happy-or-sad.zip
2.67MB [00:00, 85.8MB/s]


In [34]:
# GRADED FUNCTION: train_happy_sad_model
def train_happy_sad_model():
    # Please write your code only where you are indicated.
    # please do not remove # model fitting inline comments.

    DESIRED_ACCURACY = 0.999

    class myCallback(tf.keras.callbacks.Callback):
        
        # YOUR CODE START HERE
        def on_epoch_end(self, callback, logs={}):
            if (logs.get('accuracy')>DESIRED_ACCURACY):
              print("\n Reached 99.9%")
              self.model.stop_training = True
        # YOUR CODE END HERE
        
    callbacks = myCallback()
    
    # This Code Block should Define and Compile the Model. Please assume the images are 150 X 150 in your implementation.
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Flatten(), # for DNN
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid') # sigmoid for binary classification
    ])

    from tensorflow.keras.optimizers import RMSprop

    model.compile(optimizer=RMSprop(learning_rate=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy']
                 )
    
    # This code block should create an instance of an ImageDataGenerator called train_datagen 
    # And a train_generator by calling train_datagen.flow_from_directory

    from tensorflow.keras.preprocessing.image import ImageDataGenerator

    train_datagen = ImageDataGenerator(rescale=1/255)

    # Please use a target_size of 150 X 150.
    train_generator = train_datagen.flow_from_directory('h-or-s',
                                                        target_size = (150,150),
                                                        batch_size = 10,
                                                        class_mode = 'binary'
                                                       )
    # Expected output: 'Found 80 images belonging to 2 classes'

    # This code block should call model.fit_generator and train for
    # a number of epochs.
    # model fitting
    callbacks = myCallback()
    history = model.fit(train_generator,
                        steps_per_epoch=4,
                        epochs = 15,
                        verbose = 1,
                        callbacks=[callbacks]
                       )
    
    # verbose in fit()
    ## 0 = silent, 1 = progress bar, 2 = one line per epoch 
    
    return history.history['accuracy'][-1]

tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 120 batches). You may need to use the repeat() function when building your dataset.

steps_per_epoch * epochs가 전체 데이터 사이즈보다 커서 발생하는 오류

 step_per_epoch와 epoch와 전체 데이터 사이즈가 같도록 맞춰야 함


In [35]:
train_happy_sad_model()

Found 80 images belonging to 2 classes.
Epoch 1/15
4/4 [==============================] - 1s 29ms/step - loss: 2.6992 - accuracy: 0.5250
Epoch 2/15
4/4 [==============================] - 0s 29ms/step - loss: 0.6374 - accuracy: 0.6750
Epoch 3/15
4/4 [==============================] - 0s 31ms/step - loss: 0.5581 - accuracy: 0.7250
Epoch 4/15
4/4 [==============================] - 0s 32ms/step - loss: 0.6213 - accuracy: 0.6500
Epoch 5/15
4/4 [==============================] - 0s 27ms/step - loss: 0.5145 - accuracy: 0.7750
Epoch 6/15
4/4 [==============================] - 0s 31ms/step - loss: 0.2203 - accuracy: 0.8500
Epoch 7/15
4/4 [==============================] - 0s 28ms/step - loss: 0.1990 - accuracy: 0.8750
Epoch 8/15
4/4 [==============================] - 0s 30ms/step - loss: 0.0796 - accuracy: 1.0000

 Reached 99.9%


1.0